# 🤖 Chatbot Integrado con Comparador de PDFs# 
Este notebook integra el comparador de PDFs con diferentes tipos de chatbots:
- Chatbot de consola simple
- Chatbot web con Streamlit
- API REST con FastAPI
- Integración con WhatsApp/Telegram

## 📦 Instalaciones adicionales:
```bash
pip install streamlit fastapi uvicorn python-telegram-bot twilio
pip install gradio chainlit  # Alternativas para UI
```

## 🚀 BLOQUE 1: Importaciones y Dependencias del Chatbot

In [ ]:
import streamlit as st
import gradio as gr
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import JSONResponse
import uvicorn
import asyncio
import tempfile
import os
from typing import Dict, List, Optional, Tuple
import json
from datetime import datetime
import logging

# Para integraciones de mensajería
try:
    from telegram import Update, Bot
    from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackContext
    TELEGRAM_AVAILABLE = True
except ImportError:
    TELEGRAM_AVAILABLE = False
    print("⚠️ Telegram no disponible. Instala: pip install python-telegram-bot")

try:
    from twilio.rest import Client as TwilioClient
    TWILIO_AVAILABLE = True
except ImportError:
    TWILIO_AVAILABLE = False
    print("⚠️ Twilio no disponible. Instala: pip install twilio")

# Importar todas las funciones del comparador PDF
# (Asumiendo que el código anterior está disponible)
print("✅ Importaciones del chatbot completadas")

## 🎯 BLOQUE 2: Clase Base del Chatbot PDF

In [ ]:
# %% [markdown]
# ## 🎯 BLOQUE 2: Clase Base del Chatbot PDF

# %%
class ChatbotPDFComparator:
    """Clase base para el chatbot integrado con comparador de PDFs"""
    
    def __init__(self, llm_provider="vllm"):
        self.llm_provider = llm_provider
        self.conversation_history = []
        self.current_session = {}
        self.logger = logging.getLogger(__name__)
        
        # Estados de conversación
        self.ESTADOS = {
            "INICIO": "inicio",
            "ESPERANDO_PDF1": "esperando_pdf1", 
            "ESPERANDO_PDF2": "esperando_pdf2",
            "CONFIGURANDO_ANALISIS": "configurando_analisis",
            "PROCESANDO": "procesando",
            "MOSTRANDO_RESULTADOS": "mostrando_resultados"
        }
        
        self.estado_actual = self.ESTADOS["INICIO"]
        
    def procesar_mensaje(self, mensaje: str, archivos: List[str] = None) -> str:
        """
        Procesa un mensaje del usuario y determina la respuesta
        
        Args:
            mensaje: Texto del mensaje del usuario
            archivos: Lista de rutas de archivos subidos
            
        Returns:
            str: Respuesta del chatbot
        """
        mensaje_lower = mensaje.lower().strip()
        
        # Comandos especiales globales
        if mensaje_lower in ["/start", "/inicio", "hola", "help", "/help"]:
            return self._mostrar_ayuda()
        
        elif mensaje_lower in ["/status", "/estado"]:
            return self._mostrar_estado()
        
        elif mensaje_lower in ["/reset", "/reiniciar"]:
            return self._reiniciar_sesion()
        
        # Comandos específicos según estado
        elif self.estado_actual == self.ESTADOS["MOSTRANDO_RESULTADOS"]:
            return self._manejar_post_resultados(mensaje, archivos)
        
        # Procesamiento según estado actual
        elif self.estado_actual == self.ESTADOS["INICIO"]:
            return self._manejar_inicio(mensaje, archivos)
        
        elif self.estado_actual == self.ESTADOS["ESPERANDO_PDF1"]:
            return self._manejar_pdf1(mensaje, archivos)
        
        elif self.estado_actual == self.ESTADOS["ESPERANDO_PDF2"]:
            return self._manejar_pdf2(mensaje, archivos)
        
        elif self.estado_actual == self.ESTADOS["CONFIGURANDO_ANALISIS"]:
            return self._manejar_configuracion(mensaje)
        
        else:
            return "🤖 No entiendo tu mensaje. Escribe /help para ver los comandos disponibles."
    
    def _mostrar_ayuda(self) -> str:
        """Muestra la ayuda del chatbot"""
        self.estado_actual = self.ESTADOS["INICIO"]
        return """
🤖 **CHATBOT COMPARADOR DE PDFs**

**Comandos disponibles:**
- `/start` - Iniciar nueva comparación
- `/help` - Mostrar esta ayuda  
- `/status` - Ver estado actual
- `/reset` - Reiniciar sesión

**¿Qué puedo hacer?**
✅ Comparar dos documentos PDF
✅ Análisis con IA (vLLM, OpenAI, Ollama)
✅ Análisis especializado por dominio
✅ Generar reportes detallados

**Para empezar:**
Escribe "comparar pdfs" o sube directamente tus archivos PDF.

**Tipos de análisis disponibles:**
- **Rápido** - Resumen básico
- **Completo** - Análisis detallado
- **Legal** - Especializado en documentos legales
- **Técnico** - Para documentación técnica
- **Académico** - Para papers y artículos
        """
    
    def _mostrar_estado(self) -> str:
        """Muestra el estado actual de la conversación"""
        estados_texto = {
            "inicio": "🏁 Esperando instrucciones",
            "esperando_pdf1": "📄 Esperando primer PDF",
            "esperando_pdf2": "📄 Esperando segundo PDF", 
            "configurando_analisis": "⚙️ Configurando tipo de análisis",
            "procesando": "⏳ Procesando documentos...",
            "mostrando_resultados": "📊 Mostrando resultados"
        }
        
        estado_texto = estados_texto.get(self.estado_actual, "❓ Estado desconocido")
        
        info_sesion = ""
        if self.current_session:
            if "pdf1" in self.current_session:
                info_sesion += f"\n📄 PDF 1: {self.current_session['pdf1']['nombre']}"
            if "pdf2" in self.current_session:
                info_sesion += f"\n📄 PDF 2: {self.current_session['pdf2']['nombre']}"
        
        return f"🤖 **Estado actual:** {estado_texto}{info_sesion}"
    
    def _reiniciar_sesion(self) -> str:
        """Reinicia la sesión actual"""
        self.estado_actual = self.ESTADOS["INICIO"]
        self.current_session = {}
        return "🔄 Sesión reiniciada. ¿En qué puedo ayudarte?"
    
    def _manejar_inicio(self, mensaje: str, archivos: List[str] = None) -> str:
        """Maneja mensajes en estado inicial"""
        if archivos and len(archivos) >= 2:
            # Usuario subió 2 PDFs directamente
            return self._procesar_ambos_pdfs(archivos[:2])
        
        elif archivos and len(archivos) == 1:
            # Usuario subió 1 PDF
            self.current_session["pdf1"] = {"ruta": archivos[0], "nombre": os.path.basename(archivos[0])}
            self.estado_actual = self.ESTADOS["ESPERANDO_PDF2"]
            return f"✅ Primer PDF recibido: {self.current_session['pdf1']['nombre']}\n📄 Por favor, envía el segundo PDF para comparar."
        
        elif any(palabra in mensaje.lower() for palabra in ["comparar", "pdf", "documento", "analizar"]):
            self.estado_actual = self.ESTADOS["ESPERANDO_PDF1"]
            return "📄 Perfecto! Para comparar PDFs necesito dos documentos.\n\n🔹 Envía el **primer PDF** que quieres comparar."
        
        else:
            return "🤖 ¡Hola! Soy tu asistente para comparar documentos PDF.\n\n" + self._mostrar_ayuda()
    
    def _manejar_pdf1(self, mensaje: str, archivos: List[str] = None) -> str:
        """Maneja la recepción del primer PDF"""
        if archivos and len(archivos) >= 1:
            self.current_session["pdf1"] = {"ruta": archivos[0], "nombre": os.path.basename(archivos[0])}
            self.estado_actual = self.ESTADOS["ESPERANDO_PDF2"]
            return f"✅ Primer PDF guardado: {self.current_session['pdf1']['nombre']}\n\n📄 Ahora envía el **segundo PDF** para comparar."
        else:
            return "📄 Por favor, envía el primer archivo PDF. Puedes arrastrarlo o usar el botón de adjuntar archivo."
    
    def _manejar_pdf2(self, mensaje: str, archivos: List[str] = None) -> str:
        """Maneja la recepción del segundo PDF"""
        if archivos and len(archivos) >= 1:
            self.current_session["pdf2"] = {"ruta": archivos[0], "nombre": os.path.basename(archivos[0])}
            self.estado_actual = self.ESTADOS["CONFIGURANDO_ANALISIS"]
            return self._ofrecer_tipos_analisis()
        else:
            return "📄 Por favor, envía el segundo archivo PDF para completar la comparación."
    
    def _ofrecer_tipos_analisis(self) -> str:
        """Ofrece los tipos de análisis disponibles"""
        return f"""
✅ **PDFs recibidos:**
- 📄 PDF 1: {self.current_session['pdf1']['nombre']}
- 📄 PDF 2: {self.current_session['pdf2']['nombre']}

🎯 **¿Qué tipo de análisis quieres?**

**Escribe el número o nombre:**
1️⃣ **Rápido** - Resumen básico (2-3 min)
2️⃣ **Completo** - Análisis detallado (5-10 min)  
3️⃣ **Legal** - Especializado en documentos legales
4️⃣ **Técnico** - Para documentación técnica
5️⃣ **Académico** - Para papers y artículos científicos

O simplemente escribe: "rápido", "completo", "legal", "técnico", "académico"
        """
    
    def _manejar_configuracion(self, mensaje: str) -> str:
        """Maneja la configuración del tipo de análisis"""
        mensaje_lower = mensaje.lower().strip()
        
        # Mapear respuestas a tipos de análisis
        tipo_analisis = None
        dominio = None
        
        if mensaje_lower in ["1", "rapido", "rápido", "quick", "basico", "básico"]:
            tipo_analisis = "quick"
        elif mensaje_lower in ["2", "completo", "comprehensive", "detallado"]:
            tipo_analisis = "comprehensive"
        elif mensaje_lower in ["3", "legal", "juridico", "jurídico", "contratos"]:
            tipo_analisis = "domain"
            dominio = "documentos legales y contratos"
        elif mensaje_lower in ["4", "tecnico", "técnico", "technical", "software"]:
            tipo_analisis = "domain"
            dominio = "documentación técnica y software"
        elif mensaje_lower in ["5", "academico", "académico", "cientifico", "científico", "papers"]:
            tipo_analisis = "domain"
            dominio = "papers académicos y científicos"
        else:
            return """
❓ No entiendo el tipo de análisis. Por favor elige una opción:

1️⃣ **Rápido** 
2️⃣ **Completo**
3️⃣ **Legal**
4️⃣ **Técnico** 
5️⃣ **Académico**

Escribe el número o el nombre del análisis.
            """
        
        # Ejecutar comparación
        return self._ejecutar_comparacion(tipo_analisis, dominio)
    
    def _procesar_ambos_pdfs(self, archivos: List[str]) -> str:
        """Procesa cuando el usuario sube ambos PDFs de una vez"""
        self.current_session["pdf1"] = {"ruta": archivos[0], "nombre": os.path.basename(archivos[0])}
        self.current_session["pdf2"] = {"ruta": archivos[1], "nombre": os.path.basename(archivos[1])}
        self.estado_actual = self.ESTADOS["CONFIGURANDO_ANALISIS"]
        return self._ofrecer_tipos_analisis()
    
    def _ejecutar_comparacion(self, tipo_analisis: str, dominio: str = None) -> str:
        """Ejecuta la comparación de PDFs"""
        self.estado_actual = self.ESTADOS["PROCESANDO"]
        
        try:
            pdf1_path = self.current_session["pdf1"]["ruta"]
            pdf2_path = self.current_session["pdf2"]["ruta"]
            
            # Aquí llamarías a tu función de comparación
            # resultados = comparar_pdfs_completo(pdf1_path, pdf2_path, True, tipo_analisis, dominio)
            
            # Por ahora simulamos el resultado
            resultados_simulados = self._simular_resultados(tipo_analisis, dominio)
            
            self.current_session["resultados"] = resultados_simulados
            self.estado_actual = self.ESTADOS["MOSTRANDO_RESULTADOS"]
            
            return self._formatear_resultados_para_chat(resultados_simulados, tipo_analisis)
            
        except Exception as e:
            self.estado_actual = self.ESTADOS["INICIO"]
            return f"❌ Error procesando los PDFs: {str(e)}\n\nEscribe /help para intentar de nuevo."
    
    def _simular_resultados(self, tipo_analisis: str, dominio: str = None) -> Dict:
        """Simula resultados para demostración"""
        return {
            "puntuacion_similitud_combinada": 0.75,
            "tiempo_total_procesamiento": 45.2,
            "analisis_basico": {"porcentaje_similitud": 72.5},
            "analisis_semantico": {"porcentaje_similitud": 78.3},
            "analisis_tfidf": {"porcentaje_similitud": 71.8},
            "analisis_langchain": {
                "exito": True,
                "analisis": {
                    "puntuacion_similitud": 76,
                    "similitudes": "• Ambos documentos tratan sobre IA\n• Estructura similar\n• Vocabulario técnico común",
                    "diferencias": "• Enfoque diferente en metodología\n• Conclusiones distintas\n• Nivel de detalle variable",
                    "resumen": "Documentos relacionados con alta similitud temática pero enfoques diferentes."
                }
            },
            "tipo_analisis": tipo_analisis,
            "dominio": dominio
        }
    
    def _formatear_resultados_para_chat(self, resultados: Dict, tipo_analisis: str) -> str:
        """Formatea los resultados para mostrar en el chat"""
        similitud = resultados["puntuacion_similitud_combinada"] * 100
        tiempo = resultados["tiempo_total_procesamiento"]
        
        respuesta = f"""
🎉 **¡Análisis completado!**

📊 **Similitud general: {similitud:.1f}%**
⏱️ **Tiempo de procesamiento: {tiempo:.1f} segundos**

📈 **Puntuaciones detalladas:**
- Análisis básico: {resultados['analisis_basico']['porcentaje_similitud']:.1f}%
- Análisis semántico: {resultados['analisis_semantico']['porcentaje_similitud']:.1f}%
- Análisis TF-IDF: {resultados['analisis_tfidf']['porcentaje_similitud']:.1f}%
- Análisis IA: {resultados['analisis_langchain']['analisis']['puntuacion_similitud']}%
        """
        
        # Agregar análisis de IA si está disponible
        if resultados["analisis_langchain"]["exito"]:
            ia_analisis = resultados["analisis_langchain"]["analisis"]
            respuesta += f"""

🤖 **Análisis Inteligente:**

✅ **Similitudes:**
{ia_analisis['similitudes']}

❗ **Diferencias:**
{ia_analisis['diferencias']}

📝 **Resumen:**
{ia_analisis['resumen']}
            """
        
        respuesta += """

🎯 **¿Qué quieres hacer ahora?**
- Escribe **"reporte"** para generar reporte completo
- Escribe **"exportar"** para descargar resultados
- Escribe **"nuevo"** para comparar otros PDFs
- Escribe **"analisis"** para ver más detalles
- Escribe /help para ver más opciones
        """
        
        return respuesta
    
    def _manejar_post_resultados(self, mensaje: str, archivos: List[str] = None) -> str:
        """Maneja comandos después de mostrar resultados"""
        mensaje_lower = mensaje.lower().strip()
        
        if mensaje_lower in ["reporte", "report", "generar reporte", "reporte completo"]:
            return self._generar_reporte_detallado()
        
        elif mensaje_lower in ["exportar", "export", "descargar", "guardar"]:
            return self._exportar_resultados()
        
        elif mensaje_lower in ["nuevo", "nueva comparacion", "nueva comparación", "otro", "comparar otros"]:
            return self._iniciar_nueva_comparacion()
        
        elif mensaje_lower in ["analisis", "análisis", "detalles", "más detalles"]:
            return self._mostrar_analisis_detallado()
        
        elif mensaje_lower in ["resumen", "summary", "tldr"]:
            return self._mostrar_resumen_ejecutivo()
        
        else:
            return """
🤖 **Opciones disponibles después del análisis:**

📊 **"reporte"** - Generar reporte completo detallado
💾 **"exportar"** - Descargar resultados en JSON
🆕 **"nuevo"** - Comparar otros documentos
📈 **"analisis"** - Ver análisis más detallado
📝 **"resumen"** - Ver resumen ejecutivo

O escribe /help para ver todos los comandos.
            """
    
    def _generar_reporte_detallado(self) -> str:
        """Genera un reporte detallado de los resultados"""
        if "resultados" not in self.current_session:
            return "❌ No hay resultados disponibles para generar reporte."
        
        resultados = self.current_session["resultados"]
        
        reporte = f"""
📊 **REPORTE DETALLADO DE COMPARACIÓN**

════════════════════════════════════════

📄 **DOCUMENTOS ANALIZADOS:**
- PDF 1: {self.current_session['pdf1']['nombre']}
- PDF 2: {self.current_session['pdf2']['nombre']}

📈 **PUNTUACIONES DE SIMILITUD:**
- **Similitud General:** {resultados['puntuacion_similitud_combinada']*100:.1f}%
- **Análisis Básico:** {resultados['analisis_basico']['porcentaje_similitud']:.1f}%
- **Análisis Semántico:** {resultados['analisis_semantico']['porcentaje_similitud']:.1f}%
- **Análisis TF-IDF:** {resultados['analisis_tfidf']['porcentaje_similitud']:.1f}%
- **Análisis IA:** {resultados['analisis_langchain']['analisis']['puntuacion_similitud']}%

⏱️ **RENDIMIENTO:**
- Tiempo total: {resultados['tiempo_total_procesamiento']:.2f} segundos
- Tipo de análisis: {resultados['tipo_analisis']}
"""
        
        if resultados.get('dominio'):
            reporte += f"• Dominio especializado: {resultados['dominio']}\n"
        
        if resultados["analisis_langchain"]["exito"]:
            ia_analisis = resultados["analisis_langchain"]["analisis"]
            reporte += f"""
🤖 **ANÁLISIS INTELIGENTE DETALLADO:**

✅ **Similitudes Identificadas:**
{ia_analisis['similitudes']}

❗ **Diferencias Encontradas:**
{ia_analisis['diferencias']}

📝 **Conclusiones:**
{ia_analisis['resumen']}
            """
        
        reporte += """
════════════════════════════════════════

🎯 **RECOMENDACIONES:**
- Los documentos tienen una similitud del {:.1f}%
- {}

💡 **Próximos pasos sugeridos:**
- Escribe "exportar" para guardar estos resultados
- Escribe "nuevo" para comparar otros documentos
- Escribe /help para más opciones
        """.format(
            resultados['puntuacion_similitud_combinada']*100,
            self._interpretar_similitud(resultados['puntuacion_similitud_combinada'])
        )
        
        return reporte
    
    def _exportar_resultados(self) -> str:
        """Simula la exportación de resultados"""
        if "resultados" not in self.current_session:
            return "❌ No hay resultados disponibles para exportar."
        
        # En una implementación real, aquí guardarías los resultados
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        nombre_archivo = f"comparacion_pdfs_{timestamp}.json"
        
        return f"""
💾 **RESULTADOS EXPORTADOS**

✅ Archivo generado: `{nombre_archivo}`
📊 Datos incluidos:
- Puntuaciones de similitud
- Análisis detallado por método
- Metadatos de documentos
- Análisis de IA completo

📁 **Contenido del archivo:**
- Formato: JSON estructurado
- Tamaño: ~{len(str(self.current_session['resultados']))} caracteres
- Compatible con herramientas de análisis

💡 **¿Qué hacer ahora?**
- Escribe "nuevo" para otra comparación
- Escribe "analisis" para ver más detalles
- Escribe /help para más opciones
        """
    
    def _iniciar_nueva_comparacion(self) -> str:
        """Inicia una nueva comparación"""
        # Limpiar sesión pero mantener configuración
        self.current_session = {}
        self.estado_actual = self.ESTADOS["INICIO"]
        
        return """
🆕 **NUEVA COMPARACIÓN INICIADA**

✨ Sesión reiniciada y lista para nuevos documentos.

📄 **Para comenzar:**
- Sube dos archivos PDF, o
- Escribe "comparar pdfs" y sigue las instrucciones

🎯 **Tipos de análisis disponibles:**
- Rápido - Resumen básico
- Completo - Análisis detallado
- Legal - Documentos jurídicos
- Técnico - Documentación técnica
- Académico - Papers científicos

¿Qué documentos quieres comparar ahora?
        """
    
    def _mostrar_analisis_detallado(self) -> str:
        """Muestra análisis más detallado"""
        if "resultados" not in self.current_session:
            return "❌ No hay resultados disponibles."
        
        resultados = self.current_session["resultados"]
        
        return f"""
📈 **ANÁLISIS DETALLADO POR MÉTODO**

🔍 **ANÁLISIS BÁSICO (difflib):**
- Similitud textual: {resultados['analisis_basico']['porcentaje_similitud']:.1f}%
- Método: Comparación línea por línea
- Fortaleza: Detecta cambios exactos de texto

🧠 **ANÁLISIS SEMÁNTICO (embeddings):**
- Similitud conceptual: {resultados['analisis_semantico']['porcentaje_similitud']:.1f}%
- Método: Vectores semánticos con IA
- Fortaleza: Entiende significado y contexto

📊 **ANÁLISIS TF-IDF:**
- Similitud temática: {resultados['analisis_tfidf']['porcentaje_similitud']:.1f}%
- Método: Frecuencia de términos importantes
- Fortaleza: Identifica temas y palabras clave

🤖 **ANÁLISIS CON IA:**
- Puntuación inteligente: {resultados['analisis_langchain']['analisis']['puntuacion_similitud']}%
- Método: Modelo de lenguaje avanzado
- Fortaleza: Comprensión contextual profunda

📊 **INTERPRETACIÓN:**
{self._interpretar_similitud(resultados['puntuacion_similitud_combinada'])}

💡 **¿Necesitas más información?**
- "reporte" - Reporte completo
- "resumen" - Resumen ejecutivo
- "exportar" - Guardar resultados
        """
    
    def _mostrar_resumen_ejecutivo(self) -> str:
        """Muestra un resumen ejecutivo conciso"""
        if "resultados" not in self.current_session:
            return "❌ No hay resultados disponibles."
        
        resultados = self.current_session["resultados"]
        similitud = resultados['puntuacion_similitud_combinada'] * 100
        
        # Determinar nivel de similitud
        if similitud >= 80:
            nivel = "🟢 MUY ALTA"
            icono = "🎯"
        elif similitud >= 60:
            nivel = "🟡 ALTA"
            icono = "📊"
        elif similitud >= 40:
            nivel = "🟠 MEDIA"
            icono = "⚖️"
        else:
            nivel = "🔴 BAJA"
            icono = "📉"
        
        return f"""
📝 **RESUMEN EJECUTIVO**

{icono} **SIMILITUD {nivel}: {similitud:.1f}%**

📄 **DOCUMENTOS:**
- {self.current_session['pdf1']['nombre']}
- {self.current_session['pdf2']['nombre']}

⚡ **CONCLUSIÓN RÁPIDA:**
{self._interpretar_similitud(resultados['puntuacion_similitud_combinada'])}

🔍 **HALLAZGOS CLAVE:**
{resultados['analisis_langchain']['analisis']['resumen']}

⏱️ **Procesado en {resultados['tiempo_total_procesamiento']:.1f} segundos**

🎯 **PRÓXIMOS PASOS:**
- "reporte" → Análisis completo
- "nuevo" → Comparar otros documentos
- "exportar" → Guardar resultados
        """
    
    def _interpretar_similitud(self, similitud: float) -> str:
        """Interpreta el nivel de similitud"""
        porcentaje = similitud * 100
        
        if porcentaje >= 90:
            return "Los documentos son prácticamente idénticos o versiones muy similares."
        elif porcentaje >= 80:
            return "Los documentos son muy similares con diferencias menores."
        elif porcentaje >= 70:
            return "Los documentos comparten contenido significativo pero tienen diferencias notables."
        elif porcentaje >= 60:
            return "Los documentos tienen similitudes moderadas, posiblemente del mismo dominio."
        elif porcentaje >= 40:
            return "Los documentos tienen algunas similitudes pero son bastante diferentes."
        elif porcentaje >= 20:
            return "Los documentos tienen pocas similitudes, contenido mayormente diferente."
        else:
            return "Los documentos son muy diferentes, con similitudes mínimas."

print("✅ BLOQUE 2 - Clase base del chatbot creada")

## 💬 BLOQUE 3: Chatbot de Consola

In [ ]:
class ConsoleChatbot(ChatbotPDFComparator):
    """Chatbot de consola simple"""
    
    def __init__(self, llm_provider="vllm"):
        super().__init__(llm_provider)
        self.running = True
    
    def iniciar_chat(self):
        """Inicia el chat en consola"""
        print("🤖 " + "="*60)
        print("🤖 CHATBOT COMPARADOR DE PDFs - MODO CONSOLA")
        print("🤖 " + "="*60)
        print(self._mostrar_ayuda())
        print("\n💡 Tip: Puedes simular subida de archivos escribiendo rutas como:")
        print("   📄 /ruta/a/documento1.pdf")
        print("   📄 /ruta/a/documento2.pdf")
        print("\n🔚 Escribe 'salir' o 'quit' para terminar")
        print("-" * 60)
        
        while self.running:
            try:
                # Obtener input del usuario
                user_input = input("\n👤 Usuario: ").strip()
                
                if user_input.lower() in ['salir', 'quit', 'exit', 'bye']:
                    print("🤖 ¡Hasta luego! 👋")
                    self.running = False
                    break
                
                # Detectar si es una ruta de archivo
                archivos = self._extraer_rutas_archivos(user_input)
                
                # Procesar mensaje
                respuesta = self.procesar_mensaje(user_input, archivos)
                
                # Mostrar respuesta
                print(f"\n🤖 Bot: {respuesta}")
                
            except KeyboardInterrupt:
                print("\n\n🤖 Chat interrumpido. ¡Hasta luego! 👋")
                self.running = False
            except Exception as e:
                print(f"\n❌ Error: {str(e)}")
    
    def _extraer_rutas_archivos(self, mensaje: str) -> List[str]:
        """Extrae rutas de archivos del mensaje"""
        import re
        # Buscar patrones que parezcan rutas de archivos PDF
        patron_pdf = r'[^\s]+\.pdf'
        rutas = re.findall(patron_pdf, mensaje, re.IGNORECASE)
        
        # Verificar que los archivos existan
        rutas_validas = []
        for ruta in rutas:
            if os.path.exists(ruta):
                rutas_validas.append(ruta)
            else:
                print(f"⚠️ Archivo no encontrado: {ruta}")
        
        return rutas_validas

def ejecutar_chatbot_consola():
    """Función para ejecutar el chatbot de consola"""
    chatbot = ConsoleChatbot()
    chatbot.iniciar_chat()

print("✅ BLOQUE 3 - Chatbot de consola creado")

## 🌐 BLOQUE 4: Chatbot Web con Streamlit

In [ ]:
def crear_chatbot_streamlit():
    """Crea una interfaz web con Streamlit"""
    
    st.set_page_config(
        page_title="🤖 Chatbot PDF Comparator",
        page_icon="🤖",
        layout="wide",
        initial_sidebar_state="expanded"
    )
    
    # Sidebar para configuración
    with st.sidebar:
        st.title("⚙️ Configuración")
        
        llm_provider = st.selectbox(
            "🤖 Proveedor LLM:",
            ["vllm", "openai", "ollama"],
            index=0
        )
        
        st.markdown("---")
        st.markdown("### 📊 Estado del Chat")
        
        if 'chatbot' not in st.session_state:
            st.session_state.chatbot = ChatbotPDFComparator(llm_provider)
            st.session_state.mensajes = []
        
        estado_actual = st.session_state.chatbot.estado_actual
        st.write(f"**Estado:** {estado_actual}")
        
        if st.button("🔄 Reiniciar Chat"):
            st.session_state.chatbot = ChatbotPDFComparator(llm_provider)
            st.session_state.mensajes = []
            st.rerun()
    
    # Título principal
    st.title("🤖 Chatbot Comparador de PDFs")
    st.markdown("Chatea conmigo para comparar documentos PDF usando IA!")
    
    # Área de subida de archivos
    st.markdown("### 📄 Subir PDFs")
    col1, col2 = st.columns(2)
    
    with col1:
        uploaded_file1 = st.file_uploader(
            "📄 Primer PDF",
            type=['pdf'],
            key="pdf1"
        )
    
    with col2:
        uploaded_file2 = st.file_uploader(
            "📄 Segundo PDF", 
            type=['pdf'],
            key="pdf2"
        )
    
    # Mostrar historial de mensajes
    st.markdown("### 💬 Conversación")
    
    # Contenedor para mensajes
    chat_container = st.container()
    
    with chat_container:
        for mensaje in st.session_state.mensajes:
            if mensaje["role"] == "user":
                st.markdown(f"👤 **Usuario:** {mensaje['content']}")
            else:
                st.markdown(f"🤖 **Bot:** {mensaje['content']}")
            st.markdown("---")
    
    # Input para nuevos mensajes
    col1, col2 = st.columns([4, 1])
    
    with col1:
        user_input = st.text_input(
            "Escribe tu mensaje:",
            placeholder="Ej: 'comparar pdfs', 'análisis rápido', '/help'",
            key="user_input"
        )
    
    with col2:
        send_button = st.button("📤 Enviar", type="primary")
    
    # Procesar mensaje cuando se envía
    if send_button and user_input:
        # Agregar mensaje del usuario
        st.session_state.mensajes.append({
            "role": "user",
            "content": user_input
        })
        
        # Procesar archivos subidos
        archivos = []
        if uploaded_file1:
            # Guardar archivo temporalmente
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(uploaded_file1.getvalue())
                archivos.append(tmp_file.name)
        
        if uploaded_file2:
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(uploaded_file2.getvalue())
                archivos.append(tmp_file.name)
        
        # Obtener respuesta del chatbot
        respuesta = st.session_state.chatbot.procesar_mensaje(user_input, archivos)
        
        # Agregar respuesta del bot
        st.session_state.mensajes.append({
            "role": "bot", 
            "content": respuesta
        })
        
        # Limpiar input y recargar
        st.rerun()
    
    # Botones de acceso rápido
    st.markdown("### 🚀 Acceso Rápido")
    col1, col2, col3, col4 = st.columns(4)
    
    with col1:
        if st.button("🆘 Ayuda"):
            respuesta = st.session_state.chatbot._mostrar_ayuda()
            st.session_state.mensajes.append({"role": "bot", "content": respuesta})
            st.rerun()
    
    with col2:
        if st.button("📊 Estado"):
            respuesta = st.session_state.chatbot._mostrar_estado()
            st.session_state.mensajes.append({"role": "bot", "content": respuesta})
            st.rerun()
    
    with col3:
        if st.button("🔄 Reiniciar"):
            st.session_state.chatbot._reiniciar_sesion()
            st.session_state.mensajes = []
            st.rerun()
    
    with col4:
        if st.button("⚡ Análisis Rápido"):
            if uploaded_file1 and uploaded_file2:
                # Simular análisis rápido
                respuesta = "⚡ Iniciando análisis rápido de los PDFs subidos..."
                st.session_state.mensajes.append({"role": "bot", "content": respuesta})
                st.rerun()

def ejecutar_streamlit_app():
    """Ejecuta la aplicación Streamlit"""
    if __name__ == "__main__":
        crear_chatbot_streamlit()

print("✅ BLOQUE 4 - Chatbot Streamlit creado")

## 🚀 BLOQUE 5: API REST con FastAPI

In [ ]:
# Crear aplicación FastAPI
app = FastAPI(
    title="🤖 Chatbot PDF Comparator API",
    description="API REST para comparar documentos PDF usando un chatbot con IA",
    version="1.0.0"
)

# Almacenamiento en memoria para sesiones (en producción usar Redis/DB)
sesiones_activas = {}

@app.get("/")
async def root():
    """Endpoint raíz con información de la API"""
    return {
        "message": "🤖 Chatbot PDF Comparator API",
        "version": "1.0.0",
        "endpoints": {
            "POST /chat": "Enviar mensaje al chatbot",
            "POST /upload": "Subir archivos PDF",
            "GET /session/{session_id}": "Obtener estado de sesión",
            "DELETE /session/{session_id}": "Eliminar sesión"
        },
        "documentation": "/docs"
    }

@app.post("/chat")
async def chat_endpoint(
    session_id: str,
    message: str,
    files: List[UploadFile] = File(None)
):
    """
    Endpoint principal para chatear con el bot
    
    Args:
        session_id: ID único de la sesión
        message: Mensaje del usuario
        files: Archivos PDF opcionales
    """
    try:
        # Obtener o crear sesión
        if session_id not in sesiones_activas:
            sesiones_activas[session_id] = ChatbotPDFComparator()
        
        chatbot = sesiones_activas[session_id]
        
        # Procesar archivos si se enviaron
        rutas_archivos = []
        if files:
            for file in files:
                if file.content_type == "application/pdf":
                    # Guardar archivo temporalmente
                    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                        content = await file.read()
                        tmp_file.write(content)
                        rutas_archivos.append(tmp_file.name)
        
        # Procesar mensaje
        respuesta = chatbot.procesar_mensaje(message, rutas_archivos)
        
        return {
            "session_id": session_id,
            "user_message": message,
            "bot_response": respuesta,
            "state": chatbot.estado_actual,
            "timestamp": datetime.now().isoformat(),
            "files_received": len(rutas_archivos) if rutas_archivos else 0
        }
        
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error procesando mensaje: {str(e)}")

@app.get("/session/{session_id}")
async def get_session_status(session_id: str):
    """Obtiene el estado de una sesión"""
    if session_id not in sesiones_activas:
        raise HTTPException(status_code=404, detail="Sesión no encontrada")
    
    chatbot = sesiones_activas[session_id]
    return {
        "session_id": session_id,
        "state": chatbot.estado_actual,
        "session_data": chatbot.current_session,
        "conversation_length": len(chatbot.conversation_history)
    }

@app.delete("/session/{session_id}")
async def delete_session(session_id: str):
    """Elimina una sesión"""
    if session_id not in sesiones_activas:
        raise HTTPException(status_code=404, detail="Sesión no encontrada")
    
    del sesiones_activas[session_id]
    return {"message": f"Sesión {session_id} eliminada"}

@app.post("/upload")
async def upload_files(
    session_id: str,
    files: List[UploadFile] = File(...)
):
    """
    Endpoint para subir archivos PDF
    
    Args:
        session_id: ID de la sesión
        files: Lista de archivos PDF
    """
    try:
        if session_id not in sesiones_activas:
            sesiones_activas[session_id] = ChatbotPDFComparator()
        
        chatbot = sesiones_activas[session_id]
        rutas_guardadas = []
        
        for file in files:
            if file.content_type != "application/pdf":
                raise HTTPException(status_code=400, detail=f"Archivo {file.filename} no es PDF")
            
            # Guardar archivo
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                content = await file.read()
                tmp_file.write(content)
                rutas_guardadas.append({
                    "original_name": file.filename,
                    "temp_path": tmp_file.name,
                    "size": len(content)
                })
        
        return {
            "session_id": session_id,
            "files_uploaded": len(rutas_guardadas),
            "files": rutas_guardadas,
            "message": f"Se subieron {len(rutas_guardadas)} archivos correctamente"
        }
        
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error subiendo archivos: {str(e)}")

@app.get("/sessions")
async def list_sessions():
    """Lista todas las sesiones activas"""
    return {
        "active_sessions": len(sesiones_activas),
        "sessions": list(sesiones_activas.keys())
    }

def ejecutar_fastapi_server():
    """Ejecuta el servidor FastAPI"""
    uvicorn.run(app, host="0.0.0.0", port=8000)

print("✅ BLOQUE 5 - API FastAPI creada")

## 📱 BLOQUE 6: Integración con Telegram

In [ ]:
class TelegramChatbot(ChatbotPDFComparator):
    """Chatbot integrado con Telegram"""
    
    def __init__(self, telegram_token: str, llm_provider="vllm"):
        super().__init__(llm_provider)
        self.telegram_token = telegram_token
        self.user_sessions = {}  # Sesiones por usuario
        
        if not TELEGRAM_AVAILABLE:
            raise ImportError("python-telegram-bot no está instalado")
    
    async def start_command(self, update: Update, context: CallbackContext):
        """Maneja el comando /start"""
        user_id = update.effective_user.id
        
        if user_id not in self.user_sessions:
            self.user_sessions[user_id] = ChatbotPDFComparator(self.llm_provider)
        
        respuesta = self.user_sessions[user_id]._mostrar_ayuda()
        await update.message.reply_text(respuesta)
    
    async def handle_message(self, update: Update, context: CallbackContext):
        """Maneja mensajes de texto"""
        user_id = update.effective_user.id
        mensaje = update.message.text
        
        if user_id not in self.user_sessions:
            self.user_sessions[user_id] = ChatbotPDFComparator(self.llm_provider)
        
        chatbot = self.user_sessions[user_id]
        respuesta = chatbot.procesar_mensaje(mensaje, [])
        
        await update.message.reply_text(respuesta)
    
    async def handle_document(self, update: Update, context: CallbackContext):
        """Maneja documentos PDF"""
        user_id = update.effective_user.id
        
        if user_id not in self.user_sessions:
            self.user_sessions[user_id] = ChatbotPDFComparator(self.llm_provider)
        
        # Verificar que es un PDF
        document = update.message.document
        if not document.file_name.lower().endswith('.pdf'):
            await update.message.reply_text("❌ Solo acepto archivos PDF.")
            return
        
        # Descargar archivo
        file = await context.bot.get_file(document.file_id)
        
        # Guardar temporalmente
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
            await file.download_to_drive(tmp_file.name)
            
            chatbot = self.user_sessions[user_id]
            respuesta = chatbot.procesar_mensaje(
                f"Archivo recibido: {document.file_name}", 
                [tmp_file.name]
            )
            
            await update.message.reply_text(respuesta)
    
    def crear_aplicacion(self):
        """Crea la aplicación de Telegram"""
        application = Application.builder().token(self.telegram_token).build()
        
        # Handlers
        application.add_handler(CommandHandler("start", self.start_command))
        application.add_handler(CommandHandler("help", self.start_command))
        application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, self.handle_message))
        application.add_handler(MessageHandler(filters.Document.PDF, self.handle_document))
        
        return application
    
    def ejecutar_bot(self):
        """Ejecuta el bot de Telegram"""
        application = self.crear_aplicacion()
        print(f"🤖 Bot de Telegram iniciado...")
        application.run_polling()

def configurar_telegram_bot(token: str):
    """Configura y ejecuta el bot de Telegram"""
    if not TELEGRAM_AVAILABLE:
        print("❌ Telegram no disponible. Instala: pip install python-telegram-bot")
        return None
    
    bot = TelegramChatbot(token)
    return bot

print("✅ BLOQUE 6 - Integración Telegram creada")

## 📞 BLOQUE 7: Integración con WhatsApp (Twilio)

In [ ]:
class WhatsAppChatbot(ChatbotPDFComparator):
    """Chatbot integrado con WhatsApp usando Twilio"""
    
    def __init__(self, account_sid: str, auth_token: str, whatsapp_number: str, llm_provider="vllm"):
        super().__init__(llm_provider)
        
        if not TWILIO_AVAILABLE:
            raise ImportError("twilio no está instalado")
        
        self.client = TwilioClient(account_sid, auth_token)
        self.whatsapp_number = whatsapp_number
        self.user_sessions = {}
    
    def procesar_mensaje_whatsapp(self, from_number: str, message_body: str, media_urls: List[str] = None):
        """
        Procesa un mensaje de WhatsApp
        
        Args:
            from_number: Número del remitente
            message_body: Texto del mensaje
            media_urls: URLs de archivos adjuntos
            
        Returns:
            str: Respuesta para enviar
        """
        if from_number not in self.user_sessions:
            self.user_sessions[from_number] = ChatbotPDFComparator(self.llm_provider)
        
        chatbot = self.user_sessions[from_number]
        
        # Procesar archivos si hay URLs de media
        archivos = []
        if media_urls:
            for url in media_urls:
                # Descargar y guardar archivo (implementar según necesidades)
                # archivos.append(ruta_descargada)
                pass
        
        respuesta = chatbot.procesar_mensaje(message_body, archivos)
        return respuesta
    
    def enviar_mensaje(self, to_number: str, mensaje: str):
        """Envía un mensaje de WhatsApp"""
        try:
            message = self.client.messages.create(
                from_=f'whatsapp:{self.whatsapp_number}',
                body=mensaje,
                to=f'whatsapp:{to_number}'
            )
            return message.sid
        except Exception as e:
            print(f"Error enviando mensaje: {e}")
            return None

# FastAPI endpoints para WhatsApp webhook
@app.post("/whatsapp/webhook")
async def whatsapp_webhook(request: dict):
    """Webhook para recibir mensajes de WhatsApp"""
    try:
        # Extraer datos del webhook de Twilio
        from_number = request.get('From', '').replace('whatsapp:', '')
        message_body = request.get('Body', '')
        
        # Procesar mensaje (requiere instancia de WhatsAppChatbot)
        # respuesta = whatsapp_bot.procesar_mensaje_whatsapp(from_number, message_body)
        # whatsapp_bot.enviar_mensaje(from_number, respuesta)
        
        return {"status": "success"}
    except Exception as e:
        return {"status": "error", "message": str(e)}

print("✅ BLOQUE 7 - Integración WhatsApp creada")

## 🎨 BLOQUE 8: Interfaz con Gradio

In [ ]:
def crear_interfaz_gradio():
    """Crea una interfaz web moderna con Gradio"""
    
    # Estado global para la sesión
    chatbot_instance = ChatbotPDFComparator()
    
    def procesar_chat(message, history, pdf1, pdf2):
        """Procesa el mensaje del chat"""
        nonlocal chatbot_instance
        
        # Preparar archivos
        archivos = []
        if pdf1:
            archivos.append(pdf1.name)
        if pdf2:
            archivos.append(pdf2.name)
        
        # Obtener respuesta
        respuesta = chatbot_instance.procesar_mensaje(message, archivos)
        
        # Agregar al historial
        history.append((message, respuesta))
        
        return history, ""
    
    def reiniciar_chat():
        """Reinicia el chat"""
        nonlocal chatbot_instance
        chatbot_instance = ChatbotPDFComparator()
        return [], None, None
    
    def mostrar_estado():
        """Muestra el estado actual"""
        estado = chatbot_instance._mostrar_estado()
        return estado
    
    # Crear interfaz
    with gr.Blocks(title="🤖 Chatbot PDF Comparator", theme=gr.themes.Soft()) as demo:
        gr.Markdown("# 🤖 Chatbot Comparador de PDFs")
        gr.Markdown("Sube tus PDFs y chatea conmigo para compararlos usando IA!")
        
        with gr.Row():
            with gr.Column(scale=2):
                # Chat interface
                chatbot = gr.Chatbot(
                    height=400,
                    label="💬 Conversación",
                    bubble_full_width=False
                )
                
                with gr.Row():
                    msg = gr.Textbox(
                        placeholder="Escribe tu mensaje aquí...",
                        label="Mensaje",
                        scale=4
                    )
                    submit_btn = gr.Button("📤 Enviar", scale=1, variant="primary")
                
                with gr.Row():
                    clear_btn = gr.Button("🗑️ Limpiar Chat")
                    status_btn = gr.Button("📊 Estado")
                    help_btn = gr.Button("🆘 Ayuda")
            
            with gr.Column(scale=1):
                # File upload
                gr.Markdown("### 📄 Subir PDFs")
                pdf1 = gr.File(
                    label="📄 Primer PDF",
                    file_types=[".pdf"]
                )
                pdf2 = gr.File(
                    label="📄 Segundo PDF", 
                    file_types=[".pdf"]
                )
                
                gr.Markdown("### ⚙️ Configuración")
                llm_provider = gr.Dropdown(
                    choices=["vllm", "openai", "ollama"],
                    value="vllm",
                    label="🤖 Proveedor LLM"
                )
                
                analysis_type = gr.Dropdown(
                    choices=["rápido", "completo", "legal", "técnico", "académico"],
                    value="completo",
                    label="🎯 Tipo de Análisis"
                )
        
        # Estado del sistema
        status_output = gr.Textbox(
            label="📊 Estado del Sistema",
            interactive=False,
            max_lines=3
        )
        
        # Event handlers
        submit_btn.click(
            procesar_chat,
            inputs=[msg, chatbot, pdf1, pdf2],
            outputs=[chatbot, msg]
        )
        
        msg.submit(
            procesar_chat,
            inputs=[msg, chatbot, pdf1, pdf2],
            outputs=[chatbot, msg]
        )
        
        clear_btn.click(
            reiniciar_chat,
            outputs=[chatbot, pdf1, pdf2]
        )
        
        status_btn.click(
            mostrar_estado,
            outputs=[status_output]
        )
        
        help_btn.click(
            lambda: chatbot_instance._mostrar_ayuda(),
            outputs=[status_output]
        )
    
    return demo

def ejecutar_gradio_app():
    """Ejecuta la aplicación Gradio"""
    demo = crear_interfaz_gradio()
    demo.launch(share=True, server_name="0.0.0.0", server_port=7860)

print("✅ BLOQUE 8 - Interfaz Gradio creada")

## 🔧 BLOQUE 9: Configuración y Utilidades

In [ ]:
class ChatbotManager:
    """Manager central para todos los tipos de chatbot"""
    
    def __init__(self):
        self.chatbots_activos = {}
        self.configuracion = {
            "llm_provider": "vllm",
            "telegram_token": None,
            "whatsapp_config": None,
            "openai_api_key": None
        }
    
    def configurar(self, **kwargs):
        """Configura el manager"""
        self.configuracion.update(kwargs)
        print("✅ Configuración actualizada")
    
    def crear_chatbot_consola(self):
        """Crea chatbot de consola"""
        return ConsoleChatbot(self.configuracion["llm_provider"])
    
    def crear_chatbot_telegram(self, token: str = None):
        """Crea chatbot de Telegram"""
        token = token or self.configuracion.get("telegram_token")
        if not token:
            raise ValueError("Token de Telegram requerido")
        
        return configurar_telegram_bot(token)
    
    def crear_servidor_fastapi(self, host="0.0.0.0", port=8000):
        """Inicia servidor FastAPI"""
        print(f"🚀 Iniciando servidor FastAPI en {host}:{port}")
        uvicorn.run(app, host=host, port=port)
    
    def crear_interfaz_streamlit(self):
        """Crea interfaz Streamlit"""
        return crear_chatbot_streamlit()
    
    def crear_interfaz_gradio(self):
        """Crea interfaz Gradio"""
        return crear_interfaz_gradio()
    
    def listar_opciones(self):
        """Lista todas las opciones disponibles"""
        return """
🤖 **OPCIONES DE CHATBOT DISPONIBLES:**

1️⃣ **Consola** - Chat simple en terminal
   📝 Uso: manager.crear_chatbot_consola().iniciar_chat()

2️⃣ **Streamlit** - Interfaz web moderna
   📝 Uso: streamlit run app.py

3️⃣ **Gradio** - Interfaz web interactiva
   📝 Uso: manager.crear_interfaz_gradio().launch()

4️⃣ **FastAPI** - API REST
   📝 Uso: manager.crear_servidor_fastapi()

5️⃣ **Telegram** - Bot de Telegram
   📝 Uso: manager.crear_chatbot_telegram(token)

6️⃣ **WhatsApp** - Bot de WhatsApp (Twilio)
   📝 Uso: Configurar webhook con Twilio

🔧 **CONFIGURACIÓN:**
   manager.configurar(
       llm_provider="openai",
       telegram_token="tu_token",
       openai_api_key="tu_key"
   )
        """

# Instancia global del manager
chatbot_manager = ChatbotManager()

def mostrar_guia_configuracion():
    """Muestra guía de configuración"""
    print("""
🔧 **GUÍA DE CONFIGURACIÓN:**

1️⃣ **Configurar LLM:**
   chatbot_manager.configurar(llm_provider="openai")

2️⃣ **Telegram Bot:**
   - Crear bot con @BotFather
   - Obtener token
   - chatbot_manager.configurar(telegram_token="tu_token")

3️⃣ **WhatsApp (Twilio):**
   - Crear cuenta en Twilio
   - Configurar WhatsApp Business
   - Configurar webhook

4️⃣ **OpenAI:**
   - Obtener API key
   - chatbot_manager.configurar(openai_api_key="tu_key")

5️⃣ **vLLM Local:**
   - Instalar vLLM
   - Ejecutar servidor local
   - No requiere configuración adicional
    """)

print("✅ BLOQUE 9 - Manager y utilidades creadas")

## 🚀 BLOQUE 10: Ejemplos de Uso y Ejecución

In [ ]:
def ejemplos_uso_chatbot():
    """Muestra ejemplos de uso de cada tipo de chatbot"""
    
    print("""
🎯 **EJEMPLOS DE USO:**

1️⃣ **CHATBOT DE CONSOLA:**
   ```python
   # Ejecutar chat simple en terminal
   ejecutar_chatbot_consola()
   ```

2️⃣ **INTERFAZ WEB STREAMLIT:**
   ```python
   # En terminal:
   streamlit run tu_archivo.py
   
   # O programáticamente:
   ejecutar_streamlit_app()
   ```

3️⃣ **INTERFAZ GRADIO:**
   ```python
   demo = crear_interfaz_gradio()
   demo.launch(share=True)
   ```

4️⃣ **API REST FASTAPI:**
   ```python
   # Iniciar servidor
   ejecutar_fastapi_server()
   
   # Usar API:
   import requests
   
   response = requests.post("http://localhost:8000/chat", json={
       "session_id": "user123",
       "message": "hola"
   })
   ```

5️⃣ **BOT DE TELEGRAM:**
   ```python
   TOKEN = "tu_token_de_telegram"
   bot = configurar_telegram_bot(TOKEN)
   bot.ejecutar_bot()
   ```

6️⃣ **CONFIGURACIÓN COMPLETA:**
   ```python
   # Configurar todo
   chatbot_manager.configurar(
       llm_provider="openai",
       telegram_token="tu_token",
       openai_api_key="tu_key"
   )
   
   # Usar cualquier interfaz
   bot_consola = chatbot_manager.crear_chatbot_consola()
   bot_telegram = chatbot_manager.crear_chatbot_telegram()
   ```
    """)

def demo_interactivo():
    """Función de demostración interactiva"""
    print("🎮 **DEMO INTERACTIVO:**")
    print("=" * 50)
    
    print("¿Qué tipo de chatbot quieres probar?")
    print("1. Consola")
    print("2. Streamlit") 
    print("3. Gradio")
    print("4. FastAPI")
    print("5. Ver configuración")
    
    try:
        opcion = input("\nElige una opción (1-5): ").strip()
        
        if opcion == "1":
            print("🤖 Iniciando chatbot de consola...")
            ejecutar_chatbot_consola()
            
        elif opcion == "2":
            print("🌐 Para Streamlit, ejecuta en terminal:")
            print("   streamlit run tu_archivo.py")
            
        elif opcion == "3":
            print("🎨 Iniciando interfaz Gradio...")
            demo = crear_interfaz_gradio()
            demo.launch()
            
        elif opcion == "4":
            print("🚀 Iniciando servidor FastAPI...")
            print("   Accede a: http://localhost:8000/docs")
            ejecutar_fastapi_server()
            
        elif opcion == "5":
            mostrar_guia_configuracion()
            print(chatbot_manager.listar_opciones())
            
        else:
            print("❌ Opción no válida")
            
    except KeyboardInterrupt:
        print("\n👋 ¡Hasta luego!")
    except Exception as e:
        print(f"❌ Error: {e}")

# Funciones de inicio rápido
def inicio_rapido_consola():
    """Inicio rápido para consola"""
    print("🚀 INICIO RÁPIDO - CHATBOT CONSOLA")
    ejecutar_chatbot_consola()

def inicio_rapido_web():
    """Inicio rápido para web"""
    print("🚀 INICIO RÁPIDO - INTERFAZ WEB")
    print("Elige tu interfaz:")
    print("1. Streamlit (más completa)")
    print("2. Gradio (más visual)")
    
    opcion = input("Opción (1-2): ").strip()
    
    if opcion == "1":
        print("📝 Ejecuta: streamlit run tu_archivo.py")
    elif opcion == "2":
        demo = crear_interfaz_gradio()
        demo.launch(share=True)
    else:
        print("❌ Opción no válida")

def inicio_rapido_api():
    """Inicio rápido para API"""
    print("🚀 INICIO RÁPIDO - API REST")
    print("🌐 Iniciando servidor en http://localhost:8000")
    print("📚 Documentación en: http://localhost:8000/docs")
    ejecutar_fastapi_server()

# Mostrar ejemplos al cargar
ejemplos_uso_chatbot()
print("✅ BLOQUE 10 - Ejemplos y ejecución creados")

## 🎯 BLOQUE FINAL: Ejecución y Menu Principal

In [ ]:
def menu_principal():
    """Menú principal interactivo"""
    
    print("\n" + "="*60)
    print("🤖 CHATBOT COMPARADOR DE PDFs - MENÚ PRINCIPAL")
    print("="*60)
    
    print(chatbot_manager.listar_opciones())
    
    while True:
        print("\n🎯 **OPCIONES DISPONIBLES:**")
        print("1️⃣ Chatbot Consola")
        print("2️⃣ Interfaz Streamlit") 
        print("3️⃣ Interfaz Gradio")
        print("4️⃣ Servidor FastAPI")
        print("5️⃣ Bot Telegram")
        print("6️⃣ Configuración")
        print("7️⃣ Demo Interactivo")
        print("8️⃣ Ayuda")
        print("0️⃣ Salir")
        
        try:
            opcion = input("\n👤 Elige una opción (0-8): ").strip()
            
            if opcion == "0":
                print("👋 ¡Hasta luego!")
                break
                
            elif opcion == "1":
                inicio_rapido_consola()
                
            elif opcion == "2":
                print("📝 Para ejecutar Streamlit:")
                print("   streamlit run tu_archivo.py")
                print("   O ejecuta: ejecutar_streamlit_app()")
                
            elif opcion == "3":
                print("🎨 Iniciando Gradio...")
                demo = crear_interfaz_gradio()
                demo.launch(share=True)
                
            elif opcion == "4":
                inicio_rapido_api()
                
            elif opcion == "5":
                token = input("🔑 Token de Telegram (o Enter para omitir): ").strip()
                if token:
                    try:
                        bot = configurar_telegram_bot(token)
                        if bot:
                            bot.ejecutar_bot()
                    except Exception as e:
                        print(f"❌ Error: {e}")
                else:
                    print("⚠️ Token requerido para Telegram")
                    
            elif opcion == "6":
                mostrar_guia_configuracion()
                
            elif opcion == "7":
                demo_interactivo()
                
            elif opcion == "8":
                ejemplos_uso_chatbot()
                
            else:
                print("❌ Opción no válida")
                
        except KeyboardInterrupt:
            print("\n👋 ¡Hasta luego!")
            break
        except Exception as e:
            print(f"❌ Error: {e}")

# Función principal para ejecutar
def ejecutar_chatbot_sistema():
    """Función principal para ejecutar el sistema completo"""
    print("🚀 Iniciando Sistema de Chatbot PDF...")
    
    # Verificar dependencias
    dependencias_ok = True
    
    if not TELEGRAM_AVAILABLE:
        print("⚠️ Telegram no disponible (opcional)")
        
    if not TWILIO_AVAILABLE:
        print("⚠️ WhatsApp/Twilio no disponible (opcional)")
    
    print("✅ Sistema listo!")
    
    # Mostrar menú
    menu_principal()

print("✅ BLOQUE FINAL - Sistema completo creado")
print("\n🎉 **¡CHATBOT PDF COMPARATOR LISTO!**")
print("🚀 Ejecuta: ejecutar_chatbot_sistema() para comenzar")
print("⚡ O usa las funciones de inicio rápido:")
print("   • inicio_rapido_consola()")
print("   • inicio_rapido_web()")
print("   • inicio_rapido_api()")


In [ ]:
if __name__ == "__main__":
    # Ejecutar automáticamente
    ejecutar_chatbot_sistema()